# 강의자료의 문제: 서울시 열린데이터광장의 ```서울특별시_공공자전거 일별 대여건수_(2018~2019.03).csv```를 읽어서 데이터프레임으로 만들고 다음을 답하시오.



(1) 전/후반기의 반기별 대여건수 합계, 평균을 출력

(2) 년월별 대여건수를 계산하고 선그래프 출력

    - y축 대여건수

    - x축 년월별 (2018년 1월, 2월, ... , 12월, 2019년 1월, ... 이런 순서)

In [1]:
import os, sys
import pyspark

myConf=pyspark.SparkConf()
spark = pyspark.sql.SparkSession\
    .builder\
    .master("local")\
    .appName("myApp")\
    .config(conf=myConf)\
    .getOrCreate()

23/11/20 00:41:53 WARN Utils: Your hostname, sojaehwiui-MacBookPro.local resolves to a loopback address: 127.0.0.1; using 172.30.1.29 instead (on interface en0)
23/11/20 00:41:53 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/20 00:41:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### 데이터 전처리

In [6]:
_bicycle = spark.read.format('csv')\
    .options(header='true', inferschema='true').load('data/seoulBicycleDailyCount_2018_201903.csv')

In [7]:
bicycle=_bicycle\
    .withColumnRenamed("date", "Date")\
    .withColumnRenamed(" count", "Count")

In [8]:
bicycle=bicycle.withColumn("year", bicycle.Date.substr(1, 4))
bicycle=bicycle.withColumn("month",bicycle.Date.substr(6, 2))
bicycle.show(5)

+----------+-----+----+-----+
|      Date|Count|year|month|
+----------+-----+----+-----+
|2018-01-01| 4950|2018|   01|
|2018-01-02| 7136|2018|   01|
|2018-01-03| 7156|2018|   01|
|2018-01-04| 7102|2018|   01|
|2018-01-05| 7705|2018|   01|
+----------+-----+----+-----+
only showing top 5 rows



In [11]:
# 상반기/하반기 구분 UDF 함수 만들기
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

def classifyQuarter(s):
    s=int(s)
    if 1<=s and s< 7:
        return "1H"
    else:
        return "2H"


quarter_udf = udf(classifyQuarter, StringType())

In [12]:
# 날짜를 반기단위로 분류하여, 일 수 파악
bicycle = bicycle.withColumn("quarter", quarter_udf(bicycle.month))
bicycle.groupBy('quarter').count().show()
bicycle.show(5)

+-------+-----+
|quarter|count|
+-------+-----+
|     1H|  271|
|     2H|  184|
+-------+-----+

+----------+-----+----+-----+-------+
|      Date|Count|year|month|quarter|
+----------+-----+----+-----+-------+
|2018-01-01| 4950|2018|   01|     1H|
|2018-01-02| 7136|2018|   01|     1H|
|2018-01-03| 7156|2018|   01|     1H|
|2018-01-04| 7102|2018|   01|     1H|
|2018-01-05| 7705|2018|   01|     1H|
+----------+-----+----+-----+-------+
only showing top 5 rows



# (1) 전/후반기의 반기별 대여건수 합계, 평균을 출력

In [13]:
# 상반기/하반기 별 Count를 집계
bicycle.groupBy('quarter').agg({"count":"sum"}).show()

+-------+----------+
|quarter|sum(count)|
+-------+----------+
|     1H|   5528321|
|     2H|   6468488|
+-------+----------+



In [14]:
# 상반기/하반기 별 평균을 집계
bicycle.groupBy('quarter').agg({"count":"avg"}).show()

+-------+-----------------+
|quarter|       avg(count)|
+-------+-----------------+
|     1H|20399.70848708487|
|     2H|35154.82608695652|
+-------+-----------------+



# (2) 년월별 대여건수를 계산하고 선그래프 출력

- y축 대여건수

- x축 년월별 (2018년 1월, 2월, ... , 12월, 2019년 1월, ... 이런 순서)

In [42]:
from pyspark.sql import functions as F


_statistics = bicycle.groupBy('year', 'month').agg({"count":"sum"}).orderBy('year', 'month')
_statistics.show()

+----+-----+----------+
|year|month|sum(count)|
+----+-----+----------+
|2018|   01|    164367|
|2018|   02|    168741|
|2018|   03|    462661|
|2018|   04|    687885|
|2018|   05|    965609|
|2018|   06|   1207123|
|2018|   07|   1100015|
|2018|   08|   1037505|
|2018|   09|   1447993|
|2018|   10|   1420621|
|2018|   11|    961532|
|2018|   12|    500822|
|2019|   01|    495573|
|2019|   02|    471543|
|2019|   03|    904819|
+----+-----+----------+



In [45]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

def yearMonthConcat(year, month):
    return year + "-" + month
    
yearMonthConcatUdf = udf(yearMonthConcat, StringType())
_statistics = _statistics.withColumn("year-month", yearMonthConcatUdf(_statistics.year, _statistics.month))   
_statistics = _statistics.drop("year")
_statistics = _statistics.drop("month")
_statistics.printSchema()
_statistics.show()

root
 |-- sum(count): long (nullable = true)
 |-- year-month: string (nullable = true)

+----------+----------+
|sum(count)|year-month|
+----------+----------+
|    164367|   2018-01|
|    168741|   2018-02|
|    462661|   2018-03|
|    687885|   2018-04|
|    965609|   2018-05|
|   1207123|   2018-06|
|   1100015|   2018-07|
|   1037505|   2018-08|
|   1447993|   2018-09|
|   1420621|   2018-10|
|    961532|   2018-11|
|    500822|   2018-12|
|    495573|   2019-01|
|    471543|   2019-02|
|    904819|   2019-03|
+----------+----------+



In [1]:
import matplotlib.pyplot as plt

pandas_df=_statistics.toPandas()
plt.plot(pandas_df["year-month"], pandas_df["sum(count)"])
plt.xlabel('Year-Month')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.show()

NameError: name '_statistics' is not defined